In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
import xarray as xr

import matplotlib.pyplot as plt
import cartopy as cart

from datetime import timedelta as delta
from datetime import datetime

from parcels import (grid, Field, FieldSet, ParticleSet, JITParticle, ScipyParticle, AdvectionRK4,
                     ErrorCode, ParticleFile, Variable, plotTrajectoriesFile)

from importlib import reload
import sys
sys.path.append('/home/students/4302001/arctic-connectivity/tools')
sys.path.append('/Users/daanreijnders/surfdrive/Thesis/repo/tools')
import plot
import lifeline

In [2]:
reload(plot)
reload(lifeline)

<module 'lifeline' from '/home/students/4302001/arctic-connectivity/tools/lifeline.py'>

In [3]:
# Specify paths to velocity field and mesh
# readdir_ocean = '/Users/daanreijnders/Datasets/'
# readdir_ice = '/Users/daanreijnders/Datasets/'
# readdir_mesh = '/Users/daanreijnders/Datasets/'

readdir_ice = '/data/oceanparcels/input_data/CESM/0.1_deg/control/ice/arctic/'
readdir_ocean = '/data/oceanparcels/input_data/CESM/0.1_deg/control/ocean/arctic/'
readdir_mesh = '/scratch/DaanR/fields/'

fieldfile_ocean = 'daily_CESM_0.1degree_controlrun_year_300_arctic_region_timed.nc'
fieldfile_ice = 'monthly_icefields_CESM_0.1degree_controlrun_year_300_arctic.nc'
meshfile = 'POP_grid_coordinates.nc'

writedir = ''

In [4]:
def read_velocity_field(fieldfiles, meshfile=None):
    if not meshfile:
        meshfile = fieldfiles
    filenames = {'U': {'lon': meshfile,
                       'lat': meshfile,
                       'data':fieldfiles},
                 'V': {'lon': meshfile,
                       'lat': meshfile,
                       'data':fieldfiles}}

    variables = {'U': 'UVEL_5m',
                 'V': 'VVEL_5m'}

    dimensions = {'U': {'time': 'time',
                        'lat': 'ULAT',
                        'lon': 'ULON'},
                  'V': {'time': 'time',
                        'lat': 'ULAT',
                        'lon': 'ULON'}}
    fieldset = FieldSet.from_pop(filenames, variables, dimensions, allow_time_extrapolation=False)
    fieldset.U.vmax = 10;  fieldset.U.vmin = -10;  # set max of flow to 10 m/s
    fieldset.V.vmax = 10; fieldset.V.vmin = -10;
    return fieldset


def add_ice_fields(fieldset, fieldfile, iceVars=['aice', 'hisnap', 'hi'], meshfile=None):
    """Not working for now"""
    if not meshfile:
        meshfile = fieldfiles
    for varName in iceVars:
        filenames = {'lon': [meshfile],
                     'lat': [meshfile],
                     'data': [fieldfile]}
        variable = (varName, varName)
        dimensions = {'time': 'time',
                      'lat': 'TLAT',
                      'lon': 'TLON'}
        field = Field.from_netcdf(filenames, variable, dimensions, allow_time_extrapolation=False)
        fieldset.add_field(field)

In [5]:
fieldset = read_velocity_field(readdir_ocean+fieldfile_ocean, meshfile=readdir_mesh+meshfile)  

In [6]:
add_ice_fields(fieldset, readdir_ice+fieldfile_ice, meshfile=readdir_mesh+meshfile)

In [194]:
class myGrid:
    def __init__(self, nlon, nlat, minLat=60., maxLat=90., minLon=-180, maxLon=180):
        self.nlat = nlat
        self.nlon = nlon
        self.dlat = (maxLat - minLat)/nlat
        self.dlon = 360/nlon
        
class communityGrid(myGrid):
    def __init__(self, nlon, nlat, minLat=60., maxLat=90., minLon=-180, maxLon=180):
        myGrid.__init__(self, nlon, nlat, minLat, maxLat, minLon, maxLon)
        
        self.latOffset = self.dlat/2
        self.lonOffset = self.dlon/2
        self.latBounds = np.linspace(minLat, maxLat, nlat+1)
        self.lonBounds = np.linspace(minLon, maxLon, nlon+1)
        latarr = np.linspace(minLat+self.latOffset, maxLat-self.latOffset, nlat) # center
        lonarr = np.linspace(minLon+self.lonOffset, maxLon-self.lonOffset, nlon) # center
        self.lons, self.lats = np.meshgrid(lonarr, latarr)
        
    def countInit(self, releaseGrid):
        self.count_latlons = np.dstack((releaseGrid.lons, releaseGrid.lats))
        nPairs = releaseGrid.lons.flatten().shape[0]
        self.count_latlons = np.reshape(self.count_latlons, (nPairs, 2))
        self.initCount = np.histogram2d(self.count_latlons[:,0], self.count_latlons[:,1], bins=[self.lonBounds, self.latBounds])
        
class releaseGrid(myGrid):
    def __init__(self, nlon, nlat, release_time, minLat=60.5, maxLat=89.5, minLon=-179.5, maxLon=179.5):
        myGrid.__init__(self, nlon, nlat, minLat, maxLat, minLon, maxLon)
        if release_time:
                self.release_time = release_time
        latarr = np.linspace(minLat, maxLat, nlat) # particle position
        lonarr = np.linspace(minLon, maxLon, nlon) # particle position
        self.lons, self.lats = np.meshgrid(lonarr, latarr)

# Counter does not actually track positions

In [195]:
counter = communityGrid(60, 30)
release = releaseGrid(119, 59, 0)

In [202]:
a = np.array([[1,2,3], [4,5,6]])
b = np.array([[21,22,23],[24,25,26]])
np.dstack((a,b)).shape

(2, 3, 2)